In [1]:
from dataset.cldataset import CLDataset
from dataset.cifar10 import CIFAR10Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

/home/hamt/.conda/envs/cll_llava/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/hamt/.conda/envs/cll_llava/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
class MockConfig:
    def __init__(self):
        self.debug = True

# Example usage
cfg = MockConfig()  # Replace with actual config object

train_dataset = CIFAR10Dataset(
    root='/home/hamt/cll_vlm/cll_vlm/data/cifar10',
    train=True,
    transform=None  # No transform for visualization
)
train_dataset.cfg = cfg
original_dataset, shuffled_dataset = train_dataset.get_shuffled_labels_dataset()
print(f"Original dataset size: {len(original_dataset)}")
print(f"Shuffled dataset size: {len(shuffled_dataset)}")

Created shuffled dataset:
  - Original dataset: 50000 samples
  - Shuffled dataset: 50000 samples
  - Labels shuffled: True
Original dataset size: 50000
Shuffled dataset size: 50000


In [3]:
from models.llava_classifier import LLaVAClassifier

classifier = LLaVAClassifier(model_path="llava-hf/llava-v1.6-mistral-7b-hf")

/home/hamt/.conda/envs/cll_llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 4 files:   0%|          | 0/4 [15:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def collate_fn(batch):
    images, labels = zip(*batch)
    return list(images), torch.tensor(labels)

dataloader = DataLoader(original_dataset, batch_size=16, collate_fn=collate_fn, shuffle=False)


In [ ]:
# # Test trên 5 ảnh
# df_results = classifier.classify(original_dataset,
#                                        shuffled_dataset,
#                                        num_samples=3,
#                                        save_path="cifar10_llava_test_results.csv")

# print(df_results)

# # Hiển thị ảnh và thông tin
# for row in df_results.itertuples():
#     img, _ = original_dataset[row.index]
#     plt.imshow(img)
#     plt.title(f"Index: {row.index}\nTrue: {row.true_label} | Random: {row.random_label} | Pred: {row.predicted}")
#     plt.axis("off")
#     plt.show()

### chạy toàn bộ dataset

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    images, labels = zip(*batch)
    return list(images), torch.tensor(labels)

train_loader = DataLoader(original_dataset, batch_size=32, collate_fn=collate_fn, shuffle=False, num_workers=4)
train_loader = DataLoader(shuffled_dataset, batch_size=32, collate_fn=collate_fn, shuffle=False, num_workers=4)


In [ ]:
df_results = classifier.classify(original_dataset,
                                       shuffled_dataset,
                                    #    num_samples=100,
                                       save_path="cifar10_llava_test_results.csv")

print(df_results.head)

# Hiển thị ảnh và thông tin
def show_images_grid(df_results, original_dataset, num_images=20, per_row=5):
    rows = (num_images + per_row - 1) // per_row
    fig, axes = plt.subplots(rows, per_row, figsize=(per_row * 3.5, rows * 3.5))

    axes = axes.flatten()

    for i, row in enumerate(df_results.itertuples()):
        if i >= num_images:
            break
        img, _ = original_dataset[row.index]
        axes[i].imshow(img)
        axes[i].set_title(
            f"Idx:{row.index}\n"
            f"T:{row.true_label} | Random Label:{row.random_label}\n"
            f"Predicted:{row.predicted} | Ans:{row.raw_answer}",
            fontsize=7
        )
        axes[i].axis("off")

    # Ẩn các subplot thừa
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()

show_images_grid(df_results, original_dataset, num_images=100, per_row=5)

### Plotting

In [ ]:
def plot_random_label_distribution(csv_path):
    # Đọc file
    df = pd.read_csv(csv_path)

    # Đếm tần suất random_label
    counts = df["random_label"].value_counts()

    # Vẽ biểu đồ
    plt.figure(figsize=(10,6))
    counts.plot(kind="bar")
    plt.title("Distribution of Random Labels")
    plt.xlabel("Random Label")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
import seaborn as sns

def plot_transition_matrix(original_dataset, shuffled_dataset):
    n_classes = len(original_dataset.classes)
    matrix = np.zeros((n_classes, n_classes), dtype=int)

    for orig, shuf in zip(original_dataset.targets, shuffled_dataset.targets):
        matrix[orig, shuf] += 1

    df_matrix = pd.DataFrame(matrix, index=original_dataset.classes, columns=original_dataset.classes)

    plt.figure(figsize=(10, 8))
    sns.heatmap(df_matrix, annot=False, cmap="Blues", xticklabels=True, yticklabels=True)
    plt.title("Transition Matrix: True → Shuffled")
    plt.xlabel("Shuffled Label")
    plt.ylabel("True Label")
    plt.show()

    return df_matrix

In [ ]:
file_path = "/home/hamt/cll_vlm/cll_vlm/cifar10_llava_test_results.csv"

plot_random_label_distribution(file_path)
transition_matrix = plot_transition_matrix(original_dataset, shuffled_dataset)

In [ ]:
import pandas as pd

file_path = "/home/hamt/cll_vlm/cll_vlm/cifar10_llava_test_results.csv"
df = pd.read_csv(file_path)

cond1 = ((df["true_label"] == df["random_label"]) & (df["raw_answer"].str.lower() == "no"))

cond2 = ((df["true_label"] != df["random_label"]) & (df["raw_answer"].str.lower() == "yes"))

wrong = df[cond1 | cond2]

wrong